In [10]:
import os 
from collections import namedtuple as tup
import argparse
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from sched import SchedT1Dataset
from sched_solver import Solver
from sched_heuristic import get_rm_solution
from sched_heuristic import liu_test
import sched_heuristic as heu
import pickle
import importlib
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
### 

In [370]:
NUM_PROCS = 4
num_tasks = 16
num_proc_list = [4, 8, 12, 16]
num_tasks_list = [4, 8, 16, 32, 48, 64, 80]
with open('data/%d-%d/0.40' % (NUM_PROCS, num_tasks), 'rb') as f:
    dataset = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/4-16/0.40'

In [332]:
dataset[0][1].numpy().astype(np.int64)

array([[ 161,   11,   85],
       [  15,    1,   13],
       [  14,    2,    3],
       [ 919,   15,  851],
       [3110,  333, 2401],
       [  19,    2,    3],
       [  13,    1,    2],
       [3619,  843, 2228],
       [  19,    1,    9],
       [ 131,    5,   96],
       [1030,  212,  349],
       [  53,    3,   40],
       [  14,    1,    5],
       [1607,  872, 1260],
       [  45,    2,   10],
       [2102,  267, 1162],
       [2351,  111, 1790],
       [  94,    1,   30],
       [7930,  200,  477],
       [  25,    3,    5],
       [ 305,   18,  208],
       [  23,    1,    2],
       [  10,    1,    2],
       [2043,  261,  914],
       [3948,   74, 1434],
       [ 114,   20,   31],
       [ 146,    1,  127],
       [2558,  256, 2245],
       [  29,    3,    6],
       [9302, 1132, 2690],
       [  80,    4,   26],
       [ 116,    6,   58]])

In [368]:
importlib.reload(heu)
ress = []

def scores_to_priority(scores):
    rank = np.argsort(-scores)
    priority = np.zeros_like(rank)
    for i in range(len(priority)):
        priority[rank[i]] = i
    return priority

N = 1000
rms, rm_uss, tkcs, opas = [], [], [], []
DAS = []
RTAS = []
RTAS2 = []
opads = []
opacs = []
NUM_PROCS = 2
num_tasks = 10
xxx = "0.40"
with open('te/%d-%d/%s' % (NUM_PROCS, num_tasks, xxx), 'rb') as f:
    dataset = pickle.load(f)
for i, s_torch in dataset:
    print(i)
    # get period/exec_time
    if i >= N:
        break
    taskset = s_torch.numpy().astype(np.int64)

    scores = heu.get_DkC_scores(taskset, NUM_PROCS)
    priority = scores_to_priority(scores)

    da_res = heu.test_DA(taskset, NUM_PROCS, priority)
    DAS.append(da_res)
    
    rta_res = heu.test_RTA(taskset, NUM_PROCS, priority)
    RTAS.append(rta_res)
    
    rtares2 = heu.test_RTA_LC(taskset, NUM_PROCS, priority)
    RTAS2.append(rtares2)
    
    opa_d_res, _ = heu.OPA(taskset, NUM_PROCS, heu.test_DA)
    opads.append(opa_d_res)
    
    opa_c_rta_res, _ = heu.OPA(taskset, NUM_PROCS, heu.test_C_RTA)
    opacs.append(opa_c_rta_res)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [369]:
np.sum(DAS), np.sum(RTAS), np.sum(RTAS2), np.sum(opads), np.sum(opacs)

(987, 988, 988, 998, 998)

In [341]:
np.all(np.array(RTAS) > np.array(RTAS2))

False

In [211]:
np.all(DAS <= RTAS)

False

In [120]:
importlib.reload(heu)
ress = []
def scores_to_priority(scores):
    rank = np.argsort(-scores)
    priority = np.zeros_like(rank)
    for i in range(len(priority)):
        priority[rank[i]] = i
    return priority

N = 100
rms, rm_uss, tkcs, opas = [], [], [], []
for xxx in ["0.10", "0.20", "0.30", "0.40", "0.50", "0.60", "0.70", "0.80", "0.90", "1.00", "1.10", "1.20", 
                "1.30", "1.40", "1.50", "1.60", "1.70", "1.80", "1.90"]:

    RM_RES = []
    RM_US_RES = []
    TKC_RES = []
    OPA_RES = []
    with open('data/%d-%d/%s' % (NUM_PROCS, num_tasks, xxx), 'rb') as f:
        dataset = pickle.load(f)
    for i, s_torch in dataset:
        # get period/exec_time
        if i >= N:
            break
        taskset = s_torch.numpy()

        scores = heu.get_DM_scores(taskset, NUM_PROCS)
        priority = scores_to_priority(scores)
        res = heu.test_DA(taskset, NUM_PROCS, priority)
        RM_RES.append(res)

        scores = heu.get_DM_DS_scores(taskset, NUM_PROCS, zeta=.4)
        priority = scores_to_priority(scores)
        res = heu.test_DA(taskset, NUM_PROCS, priority)
        RM_US_RES.append(res)

        scores = heu.get_DkC_scores(taskset, NUM_PROCS, k=1.)
        priority = scores_to_priority(scores)
        res = heu.test_DA(taskset, NUM_PROCS, priority)    
        TKC_RES.append(res)

        res, priority = heu.OPA(taskset, NUM_PROCS, heu.test_DA)
        OPA_RES.append(res)
    ###
    rms.append(np.sum(RM_RES) / N)
    rm_uss.append(np.sum(RM_US_RES) / N)
    tkcs.append(np.sum(TKC_RES) / N) 
    opas.append(np.sum(OPA_RES) / N)

KeyboardInterrupt: 

In [ ]:
importlib.reload(heu)
ress = []
def scores_to_priority(scores):
    rank = np.argsort(-scores)
    priority = np.zeros_like(rank)
    for i in range(len(priority)):
        priority[rank[i]] = i
    return priority

N = 100
rms, rm_uss, tkcs, opas = [], [], [], []
for xxx in ["0.10", "0.20", "0.30", "0.40", "0.50", "0.60", "0.70", "0.80", "0.90", "1.00", "1.10", "1.20", 
                "1.30", "1.40", "1.50", "1.60", "1.70", "1.80", "1.90"]:

    RM_RES = []
    RM_US_RES = []
    TKC_RES = []
    OPA_RES = []
    with open('data/%d-%d/%s' % (NUM_PROCS, num_tasks, xxx), 'rb') as f:
        dataset = pickle.load(f)
    for i, s_torch in dataset:
        # get period/exec_time
        if i >= N:
            break
        taskset = s_torch.numpy()

        scores = heu.get_DM_scores(taskset, NUM_PROCS)
        priority = scores_to_priority(scores)
        res = heu.test_RTA(taskset, NUM_PROCS, priority)
        RM_RES.append(res)

        scores = heu.get_DM_DS_scores(taskset, NUM_PROCS, zeta=.4)
        priority = scores_to_priority(scores)
        res = heu.test_RTA(taskset, NUM_PROCS, priority)
        RM_US_RES.append(res)

        scores = heu.get_DkC_scores(taskset, NUM_PROCS, k=1.)
        priority = scores_to_priority(scores)
        res = heu.test_RTA(taskset, NUM_PROCS, priority)    
        TKC_RES.append(res)

        res, priority = heu.OPA(taskset, NUM_PROCS, heu.test_DA)
        OPA_RES.append(res)
    ###
    rms.append(np.sum(RM_RES) / N)
    rm_uss.append(np.sum(RM_US_RES) / N)
    tkcs.append(np.sum(TKC_RES) / N) 
    opas.append(np.sum(OPA_RES) / N)

In [ ]:
dataframe = {
    "DM": rms,
    "DM_DS":rm_uss,
    "DCMPO":tkcs,
    "OPA": opas,
}
df = pd.DataFrame(dataframe)
df.plot()
plt.show()

In [ ]:
tkcs